# Using Unidirectional LSTM

In [3]:
import pandas as pd

In [4]:
sd=pd.read_csv(r"C:\Users\SONY\Downloads\train.csv.zip")
sd.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
sd=sd.dropna()

In [8]:
X=sd.drop('label', axis=1)
Y=sd['label']

In [9]:
import tensorflow as tf

In [10]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [11]:
voc_size=100000

In [12]:
ping=X.copy()
ping.reset_index(inplace=True)

In [13]:
import nltk
import re
from nltk.corpus import stopwords

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SONY\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range (0,len(ping)):
    print(i)
    review=re.sub('[^a-zA-Z]','',ping['title'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [16]:
onehot_repre=[one_hot(words, voc_size) for words in corpus]
print(onehot_repre)

[[13811], [66738], [7098], [54026], [47039], [6843], [60989], [50701], [80551], [30319], [43778], [23688], [44602], [60467], [48207], [50122], [22348], [27095], [54822], [95369], [1468], [18604], [35246], [49037], [96024], [19632], [34508], [35797], [48420], [99011], [60297], [43106], [1343], [88951], [45083], [67731], [33384], [55893], [60783], [44249], [32047], [94087], [95156], [31890], [46108], [54381], [290], [93218], [69826], [84658], [82663], [86444], [92250], [35249], [73813], [13222], [51606], [25658], [379], [17135], [24113], [15660], [2127], [90955], [63659], [50725], [57707], [61706], [45450], [41271], [59227], [15106], [74521], [97235], [1767], [47561], [68055], [73694], [19412], [89134], [51914], [44944], [42341], [13856], [38240], [98714], [93303], [24401], [64720], [36801], [50612], [54894], [68987], [6142], [3852], [16277], [756], [32613], [75521], [28261], [987], [51114], [72007], [77140], [8230], [44215], [75486], [21492], [43592], [40446], [50221], [91109], [71588],

In [17]:
len(corpus)

18285

In [18]:
import numpy as np

In [19]:
sent_length = 20
embedded_docs=pad_sequences(onehot_repre, padding='pre', maxlen = sent_length)
embedded_docs

array([[    0,     0,     0, ...,     0,     0, 13811],
       [    0,     0,     0, ...,     0,     0, 66738],
       [    0,     0,     0, ...,     0,     0,  7098],
       ...,
       [    0,     0,     0, ...,     0,     0, 30765],
       [    0,     0,     0, ...,     0,     0, 18271],
       [    0,     0,     0, ...,     0,     0, 69905]])

In [20]:
dim=50

In [21]:
from tensorflow.keras.layers import Dropout
model=Sequential()
model.add(Embedding(voc_size, dim, input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 50)            5000000   
_________________________________________________________________
dropout (Dropout)            (None, 20, 50)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               60400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 5,060,501
Trainable params: 5,060,501
Non-trainable params: 0
_________________________________________________________________


In [22]:
X_final=np.array(embedded_docs)
Y_final=np.array(Y)

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test,Y_train,Y_test=train_test_split(X_final, Y_final , test_size=0.34, random_state=75)

In [24]:
model.fit(X_train, Y_train,validation_data=(X_test,Y_test), epochs=8, batch_size=640)

Epoch 1/8
19/19 [==============================] - 13s 325ms/step - loss: 0.6846 - accuracy: 0.5675 - val_loss: 0.6865 - val_accuracy: 0.5583
Epoch 2/8
19/19 [==============================] - 5s 263ms/step - loss: 0.6816 - accuracy: 0.5709 - val_loss: 0.6869 - val_accuracy: 0.5583
Epoch 3/8
19/19 [==============================] - 5s 281ms/step - loss: 0.6731 - accuracy: 0.5709 - val_loss: 0.6862 - val_accuracy: 0.5583
Epoch 4/8
19/19 [==============================] - 5s 267ms/step - loss: 0.6456 - accuracy: 0.5741 - val_loss: 0.6851 - val_accuracy: 0.5625
Epoch 5/8
19/19 [==============================] - 5s 272ms/step - loss: 0.5302 - accuracy: 0.7839 - val_loss: 0.6866 - val_accuracy: 0.5483
Epoch 6/8
19/19 [==============================] - 6s 297ms/step - loss: 0.2738 - accuracy: 0.9660 - val_loss: 0.7329 - val_accuracy: 0.5588
Epoch 7/8
19/19 [==============================] - 5s 281ms/step - loss: 0.1211 - accuracy: 0.9682 - val_loss: 0.8161 - val_accuracy: 0.5041
Epoch 8/8
19

In [25]:
Y_pred=np.argmax(model.predict(X_test), axis=-1)

In [26]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [27]:
confusion_matrix(Y_test, Y_pred)

array([[3471,    0],
       [2746,    0]], dtype=int64)

In [28]:
accuracy_score(Y_test, Y_pred)

0.5583078655299983

# Using Bidirectional LSTM

In [29]:
import pandas as pd

In [30]:
sd=pd.read_csv(r"C:\Users\SONY\Downloads\train.csv.zip")
sd.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [31]:
sd=sd.dropna()

In [32]:
X=sd.drop('label', axis=1)
Y=sd['label']

In [33]:
import tensorflow as tf

In [34]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [35]:
voc_size=100000

In [36]:
ping=X.copy()
ping.reset_index(inplace=True)

In [37]:
import nltk
import re
from nltk.corpus import stopwords

In [38]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SONY\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [39]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range (0,len(ping)):
    print(i)
    review=re.sub('[^a-zA-Z]','',ping['title'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [40]:
onehot_repre=[one_hot(words, voc_size) for words in corpus]
print(onehot_repre)

[[13811], [66738], [7098], [54026], [47039], [6843], [60989], [50701], [80551], [30319], [43778], [23688], [44602], [60467], [48207], [50122], [22348], [27095], [54822], [95369], [1468], [18604], [35246], [49037], [96024], [19632], [34508], [35797], [48420], [99011], [60297], [43106], [1343], [88951], [45083], [67731], [33384], [55893], [60783], [44249], [32047], [94087], [95156], [31890], [46108], [54381], [290], [93218], [69826], [84658], [82663], [86444], [92250], [35249], [73813], [13222], [51606], [25658], [379], [17135], [24113], [15660], [2127], [90955], [63659], [50725], [57707], [61706], [45450], [41271], [59227], [15106], [74521], [97235], [1767], [47561], [68055], [73694], [19412], [89134], [51914], [44944], [42341], [13856], [38240], [98714], [93303], [24401], [64720], [36801], [50612], [54894], [68987], [6142], [3852], [16277], [756], [32613], [75521], [28261], [987], [51114], [72007], [77140], [8230], [44215], [75486], [21492], [43592], [40446], [50221], [91109], [71588],

In [41]:
import numpy as np

In [42]:
sent_length = 20
embedded_docs=pad_sequences(onehot_repre, padding='pre', maxlen = sent_length)
embedded_docs

array([[    0,     0,     0, ...,     0,     0, 13811],
       [    0,     0,     0, ...,     0,     0, 66738],
       [    0,     0,     0, ...,     0,     0,  7098],
       ...,
       [    0,     0,     0, ...,     0,     0, 30765],
       [    0,     0,     0, ...,     0,     0, 18271],
       [    0,     0,     0, ...,     0,     0, 69905]])

In [43]:
dim=50

In [44]:
from tensorflow.keras.layers import Dropout
model=Sequential()
model.add(Embedding(voc_size, dim, input_length=sent_length))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 50)            5000000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 50)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               120800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 5,121,001
Trainable params: 5,121,001
Non-trainable params: 0
_________________________________________________________________


In [45]:
X_final=np.array(embedded_docs)
Y_final=np.array(Y)

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test,Y_train,Y_test=train_test_split(X_final, Y_final , test_size=0.34, random_state=75)

In [47]:
model.fit(X_train, Y_train,validation_data=(X_test,Y_test), epochs=8, batch_size=640)

Epoch 1/8
19/19 [==============================] - 19s 634ms/step - loss: 0.6851 - accuracy: 0.5666 - val_loss: 0.6864 - val_accuracy: 0.5583
Epoch 2/8
19/19 [==============================] - 10s 508ms/step - loss: 0.6824 - accuracy: 0.5709 - val_loss: 0.6865 - val_accuracy: 0.5583
Epoch 3/8
19/19 [==============================] - 9s 496ms/step - loss: 0.6799 - accuracy: 0.5709 - val_loss: 0.6860 - val_accuracy: 0.5583
Epoch 4/8
19/19 [==============================] - 10s 526ms/step - loss: 0.6701 - accuracy: 0.5710 - val_loss: 0.6850 - val_accuracy: 0.5583
Epoch 5/8
19/19 [==============================] - 10s 509ms/step - loss: 0.6292 - accuracy: 0.6235 - val_loss: 0.6880 - val_accuracy: 0.5651
Epoch 6/8
19/19 [==============================] - 10s 532ms/step - loss: 0.4730 - accuracy: 0.8435 - val_loss: 0.6888 - val_accuracy: 0.5683
Epoch 7/8
19/19 [==============================] - 10s 544ms/step - loss: 0.2199 - accuracy: 0.9643 - val_loss: 0.9556 - val_accuracy: 0.5487
Epoch 8